---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Load the university town data
    uni_town = pd.read_fwf('university_towns.txt', header=None)
    uni_state_region = pd.DataFrame(columns = ['State', 'RegionName'])
    #print(uni_state_region)
    
    # Pre-process the university town data
    uni_town[0].replace('\[.*', '', inplace=True,regex=True)
    uni_town[0].replace('\(.*', '', inplace=True,regex=True) 
    
    # Loop over the data to create the dataframe in required format
    uni_state =None
    hold_state = None
    for item in uni_town[0].iteritems():
        if item[1] in states.values():
            uni_state = item[1]
            if uni_state != hold_state:
                hold_state = uni_state
        else:
            uni_state_region = uni_state_region.append({'State': hold_state.strip(), 'RegionName':item[1].strip() }, ignore_index = True)
                            
    # View the data - unit testing
    #print(uni_state_region.iloc[184])
        
    return uni_state_region

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


Preprocess the GDP data
- Get only the quarter and the chained value in 2009 dollars
- Drop all teh NN values
- Rename the first column to Quarter
- Filter only the GDP data from 2000

In [4]:
def gdp_data_preprocessing():
    
    '''Return the preprocessed GDP data '''
    
    gdp = pd.read_excel('gdplev.xls', header = 4,  parse_cols = [4,6])
    gdp.dropna(how='all', inplace=True)
    gdp.rename(columns={'Unnamed: 0':'Quarter'}, inplace=True)
    
    # Filter the rows from 2000q1
    gdp_data = gdp[gdp['Quarter']> '1999q4']
    
    return gdp_data

In [5]:
def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # Get the preprocessed GDP data
    gdp_from_2k = gdp_data_preprocessing()
    
    # View data
    #print(gdp_from_2k[gdp_from_2k['Quarter']> '2007q4'])
    
    # Loop through the dataframe 
    for i in range(0,len(gdp_from_2k)-2):
        
        if (gdp_from_2k.iloc[i+1, 1] < gdp_from_2k.iloc[i, 1] ) & (gdp_from_2k.iloc[i+2, 1] < gdp_from_2k.iloc[i+1, 1] ):
            recession_starts = gdp_from_2k.iloc[i+1, 0]
            break  
            
    return recession_starts

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # Get the preprocessed GDP data
    gdp_from_2k = gdp_data_preprocessing()
    
    # Get the quarter in which recession starts
    recession_starts = get_recession_start()
    
    # Filter the rows after recession starts
    gdp_data_af_res_start = gdp_from_2k[gdp_from_2k['Quarter']> recession_starts]
    
    # View data
    #print(gdp_data_af_res_start)
    
    # Loop through the dataframe 
    for i in range(0,len(gdp_data_af_res_start)-2):
        
        if (gdp_data_af_res_start.iloc[i+1, 1] > gdp_data_af_res_start.iloc[i, 1] ) & (gdp_data_af_res_start.iloc[i+2, 1] > gdp_data_af_res_start.iloc[i+1, 1] ):
            recession_ends = gdp_data_af_res_start.iloc[i+2, 0]
            break  
            
    return recession_ends
       
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Get the preprocessed GDP data
    gdp_from_2k = gdp_data_preprocessing()
    
    # Get the quarter on which the recession starts
    recession_starts = get_recession_start()
    
    # Get the quarter on which the recession ends
    recession_ends = get_recession_end()
    
    # GDP data during the recession period
    recession_gdp = gdp_from_2k[(gdp_from_2k['Quarter']>= recession_starts)&(gdp_from_2k['Quarter'] <= recession_ends)]
    
    # Get the minimum GDP value
    recession_bottom = recession_gdp.loc[recession_gdp.iloc[:,1].argmin()]['Quarter']
        
    return recession_bottom

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Read the csv with only selected columns 
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv', usecols= [1,2, *range(51,251)])
    
    # Update teh state code with state name from teh dictionary
    for idx, row in housing_data.iterrows():
        housing_data.set_value(idx,'State',states[row['State']])
        
    # Create the multi-index
    housing_data.set_index(['State','RegionName'], inplace = True)
    housing_data.sortlevel(inplace=True)
    
    # View the data
    #print(housing_data.index)
    
    # Create the new dataframe with same multi-index and empty columns
    house_quar_data = pd.DataFrame(index= housing_data.index)
    
    # Get the number od columns and check whether it could divided into quarters
    n_cols = len(housing_data.columns)
    reminder = n_cols%3
    
    # Loop over the columns and calculate the mean housing proces fro the quarter
    for i in range(0,n_cols-reminder):
        old_col_name = housing_data.iloc[:,i].name
        if ('-03' in old_col_name):
            new_col_name = old_col_name.split('-')[0]+'q1'
            house_quar_data[new_col_name]= (housing_data.iloc[:,i] + housing_data.iloc[:,i-1] + housing_data.iloc[:,i-2])/3
        if ('-06' in old_col_name):
            new_col_name = old_col_name.split('-')[0]+'q2'
            house_quar_data[new_col_name]= (housing_data.iloc[:,i] + housing_data.iloc[:,i-1] + housing_data.iloc[:,i-2])/3
        if ('-09' in old_col_name):
            new_col_name = old_col_name.split('-')[0]+'q3'
            house_quar_data[new_col_name]= (housing_data.iloc[:,i] + housing_data.iloc[:,i-1] + housing_data.iloc[:,i-2])/3
        if ('-12' in old_col_name):
            new_col_name = old_col_name.split('-')[0]+'q4'
            house_quar_data[new_col_name]= (housing_data.iloc[:,i] + housing_data.iloc[:,i-1] + housing_data.iloc[:,i-2])/3
    
    if reminder != 0:
        old_col_name = house_quar_data.iloc[:,-1].name
        new_col_name = old_col_name.split('-')[0]+'q3'
        house_quar_data[new_col_name]= (housing_data.iloc[:,-1] + housing_data.iloc[:,-2])/2

    return house_quar_data

#convert_housing_data_to_quarters()
convert_housing_data_to_quarters().loc["New Jersey"].loc["Franklin Township"]
#convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

202266.66666666666

In [33]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Initialize the return values 
    better = "non-university town"
    different = False
    
    # Get the housing data
    housing_data = convert_housing_data_to_quarters()
    
    # Get the recession_bottom
    recession_bottom = get_recession_bottom()
    
    # Get the quarter before recession start
    recession_start = get_recession_start()
    quarter_bf_res_start_idx = housing_data.columns.get_loc(recession_start)-1
    quarter_bf_res_start = housing_data.columns[quarter_bf_res_start_idx]
    
    # Find the price ratio before the recession starts and the recession bottom
    housing_data['price_ratio'] = housing_data[quarter_bf_res_start]/housing_data[recession_bottom]
    
    # Filter only the price_ratio cloumn with the multi-index from housing_data
    housing_data_res = housing_data.filter(['price_ratio'])
    
    # Get the university region data
    uni_state_region =  get_list_of_university_towns()
    
    # Create Multi-index for the university state region
    uni_state_region.reset_index(drop=True, inplace=True)
    uni_state_region.set_index(['State','RegionName'], inplace = True)
    
    # Create empty dataframe for the non-university towns
    my_index = pd.MultiIndex(levels=[[0],[1]], labels=[[],[]], names=['State','RegionName'])
    non_university_towns = pd.DataFrame( index = my_index, columns=['price_ratio'])
    
    # Filter only the university town housing data
    university_towns = pd.merge(housing_data_res,uni_state_region, left_index=True, right_index=True,how='inner')
    
    # Check the length
    #print("len1 ", len(housing_data_res.index))
    #print("len2 ", len(university_towns.index))

    # Filter the non-university town housing data
    for idx, row in housing_data_res.iterrows():
        if (idx not in university_towns.index):
            non_university_towns =non_university_towns.append(row)
            
    # Check the length        
    print("len3 ", len(non_university_towns.index))
    
    # Calculate the p-value
    stat, p_value = ttest_ind(university_towns.dropna(), non_university_towns.dropna())
    
    # Check the p-Values
    #print('p-value ', p_value)
    #print("university_towns_mean = ",university_towns['price_ratio'].mean())
    #print("non_university_towns_mean = ",non_university_towns['price_ratio'].mean())

    # Calculate the 'different' value from the p-value
    if p_value < 0.01:
        different = True
    
    # Calculate the value for 'better' from the mean values
    if university_towns['price_ratio'].mean()< non_university_towns['price_ratio'].mean():
        better = "university town"
        
    # Formulate the result as a tuple    
    result = (different, p_value[0], better)
    
    return result

run_ttest()

len3  10461


(True, 0.0027134586648562414, 'university town')

In [32]:
# To debug
print(get_list_of_university_towns().groupby('State').size())

State
Alabama            8
Alaska             1
Arizona            3
Arkansas           8
California        25
Colorado           9
Connecticut        7
Delaware           2
Florida           12
Georgia           16
Hawaii             1
Idaho              3
Illinois          10
Indiana           13
Iowa              13
Kansas             6
Kentucky          11
Louisiana          8
Maine              9
Maryland          10
Massachusetts     19
Michigan          20
Minnesota         14
Mississippi        5
Missouri           9
Montana            3
Nebraska           7
Nevada             2
New Hampshire      7
New Jersey        10
New Mexico         5
New York          31
North Carolina    18
North Dakota       2
Ohio              22
Oklahoma          11
Oregon            10
Pennsylvania      43
Rhode Island       2
South Carolina    11
South Dakota       4
Tennessee         13
Texas             22
Utah               6
Vermont            6
Virginia          12
Washington         5
West Vi